<img src="../../static/aeropython_name_mini.png" alt="AeroPython" style="width: 300px;"/>

## Interactuar con arduino desde un Notebook

Importamos las librerías que vamos a usar

In [ ]:
import serial
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

Esta función nos servirá para calcular el tiempo que ha pasado desde el inicio

In [ ]:
def deltatime():
    tiempo = datetime.datetime.now()
    deltat = tiempo - starttime
    return deltat.seconds + deltat.microseconds/1000000

Esta función decodifica los mensajes que envía el arduino

In [ ]:
def decode_medida(lectura):
    _pos = lectura.find('posicion') + 10
    _pos2 = lectura.find(' ',_pos )
    medida_1 = float(lectura[_pos : _pos2])

    _pos = lectura.find('velocidad') + 11
    _pos2 = lectura.find(' ',_pos )
    medida_2 = float(lectura[_pos : _pos2])
    
    return (medida_1, medida_2)

Esta función pregunta al arduino su estado y guarda los resultados

In [ ]:
def com_status(tiempos, posiciones, velocidades, serial_com):
    serial_com.write(b'status')
    lectura = serial_com.readline().decode("utf-8")
    print(lectura) #Espera hasta que empiece a comunicarse
    if lectura[:22] == 'Modo escucha. Medida: ':
        tiempos.append(deltatime())
        medida = float(lectura[22:])
        posiciones.append(medida)
        velocidades.append(0.0)

Esta función inicia la comunicación

In [ ]:
def com_start_full():
    ser = serial.Serial('COM6', 9600) #Iniciamos comunicación
    #Linux: ser = serial.Serial('/dev/ttyACM0', 9600)
    ser.readline()
    print('Calibrando posiciones\r\n')
    ser.write(b'calibrate')
    ser.readline()
    starttime = datetime.datetime.now()    
    print('preparado\r\n')
    return ser, starttime

Esta función envía al arduino la orden de moverse

In [ ]:
def com_move(tiempos, posiciones, velocidades, serial_com, destino, medida):
    destino = str(destino).encode()
    serial_com.write(b'move')
    serial_com.readline()
    serial_com.write(destino) #Enviamos una orden
    print('mensaje enviado\r\n')
    posiciones.append(medida)
    velocidades.append(0.0)       
    tiempos.append(deltatime())

Esta función devuelve la mesa a la posición central y finaliza la comunicación

In [ ]:
def com_close(tiempos, posiciones, velocidades, serial_com,):
    destino = str(90).encode()
    serial_com.write(b'move')
    serial_com.readline()
    serial_com.write(destino) #Enviamos una orden
    print('Volviendo a posición central\r\n')
    for i in range(100): #Leemos hasta que pase a 'Esperando', o hasta que pasen 100 pasos
        finish = com_read_move(tiempos, posiciones, velocidades, [], ser)
        if finish: break
    serial_com.close()
    print ('\r\nOperación terminada')

Esta función comprueba si nos seguimos moviendo o ya hemos alcanzado el destino

In [ ]:
def com_read_move(tiempos, posiciones, velocidades, hitos, serial_com):
    finish = False
    lectura = serial_com.readline().decode("utf-8")
    print(lectura, end = '') 
    if lectura == 'Alcanzado\r\n':
        hitos.append(deltatime())
        finish = True
        print('')
    if lectura[:8] == 'moviendo':
        medida = decode_medida(lectura)
        posiciones.append(medida[0])
        velocidades.append(medida[1])        
        tiempos.append(deltatime())
    return finish

Creamos un error personalizado para detener el código de manera segura si el arduino no se estabiliza

In [ ]:
class ArduinoMovementError(Exception):
    """Exception raised for errors in the Arduino movement.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message):
        self.message = message


## Código central

In [ ]:
tiempos = []
posiciones =[]
velocidades = []
hitos = []
destinos = [15,90,120]

ser, starttime = com_start_full()

for destino in destinos:
    # -------------------------------
    com_status(tiempos, posiciones, velocidades, ser)
    # -----------------------------------

    com_move(tiempos, posiciones, velocidades, ser, destino, posiciones[-1])

    for i in range(100): #Leemos hasta que pase a 'Esperando', o hasta que pasen 100 pasos
        finish = com_read_move(tiempos, posiciones, velocidades, hitos, ser)
        if finish: break
    else:
        ser.close()
        raise ArduinoMovementError('La plataforma no alcanzó el destino en el tiempo dado')
            
com_close(tiempos, posiciones, velocidades, ser)

In [ ]:
plt.figure(figsize=(12,8))


plt.plot(tiempos, posiciones, label = 'Posición')
plt.plot(tiempos, np.array(velocidades)*100, label = 'Velocidad (x100)')
plt.hlines(0, 0, tiempos[-1], 'k','--')
for hito in hitos:
    plt.vlines(hito, -100,170, 'r', '--')
plt.legend(loc = 2)
plt.fill_between([hito,tiempos[-1]],[-100,-100],[170,170], alpha = 0.2, color = 'k')

In [ ]:
tiempos

In [ ]:
posiciones